# US Census Bureau Data Clean

In [11]:
import pandas as pd 
import numpy as np 
import os
import re 

pd.set_option('display.max_columns', None)

In [4]:
path = '/Users/tj/trevorj@berkeley.edu - Google Drive/My Drive/Classes/W210_capstone/W210 Capstone/Data/population/ACS/raw_census_bureau'
data_files = [i for i in os.listdir(path) if re.search('Data.csv', i)]
data_files

['ACSST5Y2012.S0101-Data.csv',
 'ACSST5Y2011.S0101-Data.csv',
 'ACSST5Y2014.S0101-Data.csv',
 'ACSST5Y2013.S0101-Data.csv',
 'ACSST5Y2016.S0101-Data.csv',
 'ACSST5Y2015.S0101-Data.csv',
 'ACSST5Y2017.S0101-Data.csv',
 'ACSST5Y2019.S0101-Data.csv',
 'ACSST5Y2018.S0101-Data.csv',
 'ACSST5Y2020.S0101-Data.csv']

The following code works to clean any census data <= 2016. Starting 2017, there are many more fields, and it seems field names change. I will fix this later. 

In [86]:
file_num = 5
df = pd.read_csv(os.path.join(path, data_files[file_num]))
df

,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C01_001MA,S0101_C01_001EA,S0101_C02_001E,S0101_C02_001EA,S0101_C02_001M,S0101_C02_001MA,S0101_C03_001E,S0101_C03_001M,S0101_C03_001MA,S0101_C03_001EA,S0101_C01_002E,S0101_C01_002M,S0101_C01_002MA,S0101_C01_002EA,S0101_C02_002E,S0101_C02_002EA,S0101_C02_002M,S0101_C02_002MA,S0101_C03_002E,S0101_C03_002M,S0101_C03_002MA,S0101_C03_002EA,S0101_C01_003E,S0101_C01_003EA,S0101_C01_003M,S0101_C01_003MA,S0101_C02_003E,S0101_C02_003EA,S0101_C02_003M,S0101_C02_003MA,S0101_C03_003E,S0101_C03_003M,S0101_C03_003MA,S0101_C03_003EA,S0101_C01_004E,S0101_C01_004EA,S0101_C01_004M,S0101_C01_004MA,S0101_C02_004E,S0101_C02_004M,S0101_C02_004MA,S0101_C02_004EA,S0101_C03_004E,S0101_C03_004M,S0101_C03_004MA,S0101_C03_004EA,S0101_C01_005E,S0101_C01_005M,S0101_C01_005MA,S0101_C01_005EA,S0101_C02_005E,S0101_C02_005M,S0101_C02_005MA,S0101_C02_005EA,S0101_C03_005E,S0101_C03_005EA,S0101_C03_005M,S0101_C03_005MA,S0101_C01_006E,S0101_C01_006M,S0101_C01_006MA,S0101_C01_006EA,S0101_C02_006E,S0101_C02_006EA,S0101_C02_006M,S0101_C02_006MA,S0101_C03_006E,S0101_C03_006M,S0101_C03_006MA,S0101_C03_006EA,S0101_C01_007E,S0101_C01_007M,S0101_C01_007MA,S0101_C01_007EA,S0101_C02_007E,S0101_C02_007EA,S0101_C02_007M,S0101_C02_007MA,S0101_C03_007E,S0101_C03_007M,S0101_C03_007MA,S0101_C03_007EA,S0101_C01_008E,S0101_C01_008M,S0101_C01_008MA,S0101_C01_008EA,S0101_C02_008E,S0101_C02_008EA,S0101_C02_008M,S0101_C02_008MA,S0101_C03_008E,S0101_C03_008EA,S0101_C03_008M,S0101_C03_008MA,S0101_C01_009E,S0101_C01_009EA,S0101_C01_009M,S0101_C01_009MA,S0101_C02_009E,S0101_C02_009M,S0101_C02_009MA,S0101_C02_009EA,S0101_C03_009E,S0101_C03_009EA,S0101_C03_009M,S0101_C03_009MA,S0101_C01_010E,S0101_C01_010EA,S0101_C01_010M,S0101_C01_010MA,S0101_C02_010E,S0101_C02_010M,S0101_C02_010MA,S0101_C02_010EA,S0101_C03_010E,S0101_C03_010EA,S0101_C03_010M,S0101_C03_010MA,S0101_C01_011E,S0101_C01_011EA,S0101_C01_011M,S0101_C01_011MA,S0101_C02_011E,S0101_C02_011EA,S0101_C02_011M,S0101_C02_011MA,S0101_C03_011E,S0101_C03_011EA,S0101_C03_011M,S0101_C03_011MA,S0101_C01_012E,S0101_C01_012M,S0101_C01_012MA,S0101_C01_012EA,S0101_C02_012E,S0101_C02_012EA,S0101_C02_012M,S0101_C02_012MA,S0101_C03_012E,S0101_C03_012M,S0101_C03_012MA,S0101_C03_012EA,S0101_C01_013E,S0101_C01_013M,S0101_C01_013MA,S0101_C01_013EA,S0101_C02_013E,S0101_C02_013EA,S0101_C02_013M,S0101_C02_013MA,S0101_C03_013E,S0101_C03_013M,S0101_C03_013MA,S0101_C03_013EA,S0101_C01_014E,S0101_C01_014EA,S0101_C01_014M,S0101_C01_014MA,S0101_C02_014E,S0101_C02_014M,S0101_C02_014MA,S0101_C02_014EA,S0101_C03_014E,S0101_C03_014M,S0101_C03_014MA,S0101_C03_014EA,S0101_C01_015E,S0101_C01_015M,S0101_C01_015MA,S0101_C01_015EA,S0101_C02_015E,S0101_C02_015M,S0101_C02_015MA,S0101_C02_015EA,S0101_C03_015E,S0101_C03_015EA,S0101_C03_015M,S0101_C03_015MA,S0101_C01_016E,S0101_C01_016EA,S0101_C01_016M,S0101_C01_016MA,S0101_C02_016E,S0101_C02_016M,S0101_C02_016MA,S0101_C02_016EA,S0101_C03_016E,S0101_C03_016EA,S0101_C03_016M,S0101_C03_016MA,S0101_C01_017E,S0101_C01_017M,S0101_C01_017MA,S0101_C01_017EA,S0101_C02_017E,S0101_C02_017EA,S0101_C02_017M,S0101_C02_017MA,S0101_C03_017E,S0101_C03_017M,S0101_C03_017MA,S0101_C03_017EA,S0101_C01_018E,S0101_C01_018M,S0101_C01_018MA,S0101_C01_018EA,S0101_C02_018E,S0101_C02_018EA,S0101_C02_018M,S0101_C02_018MA,S0101_C03_018E,S0101_C03_018M,S0101_C03_018MA,S0101_C03_018EA,S0101_C01_019E,S0101_C01_019M,S0101_C01_019MA,S0101_C01_019EA,S0101_C02_019E,S0101_C02_019EA,S0101_C02_019M,S0101_C02_019MA,S0101_C03_019E,S0101_C03_019EA,S0101_C03_019M,S0101_C03_019MA,S0101_C01_020E,S0101_C01_020M,S0101_C01_020MA,S0101_C01_020EA,S0101_C02_020E,S0101_C02_020M,S0101_C02_020MA,S0101_C02_020EA,S0101_C03_020E,S0101_C03_020EA,S0101_C03_020M,S0101_C03_020MA,S0101_C01_021E,S0101_C01_021M,S0101_C01_021MA,S0101_C01_021EA,S0101_C02_021E,S0101_C02_021M,S0101_C02_021MA,S0101_C02_021EA,S0101_C03_021E,S0101_C03_021EA,S0101_C03_021M,S0101_C03_021MA,S0101_C01_022E,S0101_C01_022EA,S0101_C01_022M,S0101_C01_022MA,S0101_C02_022E,S010

In [88]:
if int(data_files[file_num][7:11]) <= 2016:
  # remove first row which contains field names
  df2 = df.iloc[1:]

  # add data year in cleaner format
  df2['year'] = int(data_files[file_num][7:11])

  # give fields intuitive names
  df2 = df2.rename(columns = {
    'NAME': 'zip'
    ,'S0101_C01_001E': 'total_population'
    ,'S0101_C02_001E': 'total_population_male'
    ,'S0101_C03_001E': 'total_population_female'

    ,'S0101_C01_002E': 'population_0_4'
    ,'S0101_C02_002E': 'population_0_4_male'
    ,'S0101_C03_002E': 'population_0_4_female'

    ,'S0101_C01_003E': 'population_5_9'
    ,'S0101_C02_003E': 'population_5_9_male'
    ,'S0101_C03_003E': 'population_5_9_female'

    ,'S0101_C01_004E': 'population_10_14'
    ,'S0101_C02_004E': 'population_10_14_male'
    ,'S0101_C03_004E': 'population_10_14_female'

    ,'S0101_C01_005E': 'population_15_19'
    ,'S0101_C02_005E': 'population_15_19_male'
    ,'S0101_C03_005E': 'population_15_19_female'
    })

  # select only important fields
  df2 = df2[['year', 'zip', 
    'total_population', 'total_population_male', 'total_population_female', 
    'population_0_4', 'population_0_4_male', 'population_0_4_female',
    'population_5_9', 'population_5_9_male', 'population_5_9_female',
    'population_10_14', 'population_10_14_male', 'population_10_14_female',
    'population_15_19', 'population_15_19_male', 'population_15_19_female'
    ]]

  # extract 5-digit zip
  df2['zip'] = df2['zip'].map(lambda x: int(x[6:]))

  # some columns put "-" instead of 0
  for col in df2.columns[2:]:
    df2[col] = df2[col].map(lambda x: re.sub('-', '0', x))

  # convert measurement columns to float 
  df2.astype({i:'float' for i in df2.columns[2:]})

df2

<ipython-input-88-26b12fe93514>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['year'] = int(data_files[file_num][7:11])


,year,zip,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female
1,2015,89010,549,220,329,1.1,0.0,1.8,6.0,7.3,5.2,4.7,2.7,6.1,2.2,5.5,0.0
2,2015,89019,1804,868,936,0.7,1.4,0.0,4.8,0.0,9.3,2.3,0.7,3.8,3.0,1.8,4.2
3,2015,89060,9587,4824,4763,3.9,4.0,3.8,5.7,4.2,7.2,3.8,5.2,2.3,5.4,6.2,4.6
4,2015,89061,5337,2583,2754,4.8,4.8,4.8,3.8,6.3,1.5,5.8,6.7,4.9,3.3,0.9,5.6
5,2015,89439,1609,787,822,3.5,4.7,2.3,3.4,2.3,4.5,2.3,2.3,2.3,4.6,7.0,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,2015,96148,853,471,382,11.4,13.4,8.9,7.2,9.8,3.9,3.6,2.1,5.5,3.0,3.2,2.9
1766,2015,96150,28912,15069,13843,5.9,5.6,6.2,5.9,6.3,5.6,4.2,4.1,4.4,6.1,5.9,6.3
1767,2015,96155,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1768,2015,96161,18256,9774,8482,5.7,4.3,7.3,6.5,7.2,5.6,7.5,7.7,7.2,5.3,6.0,4.4
